### santander-customer-transaction-prediction 

#### Artificial Neural Network has been implemented using Keras and Tensorflow.

### Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler

#### Importing Data

In [3]:
df = pd.read_csv("C:\\Users\\AJIT\\Desktop\\santander-customer-transaction-prediction\\train.csv")
df_test = pd.read_csv("C:\\Users\\AJIT\\Desktop\\santander-customer-transaction-prediction\\test.csv")

In [4]:
df.shape

(200000, 202)

In [5]:
df_test.shape

(200000, 201)

In [6]:
df['Type']='Train'
df_test['Type']='Test'
data = pd.concat([df,df_test],axis=0)

C:\Users\AJIT\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
ID_col = ['ID_code']
target_col = ["target"]
other_col=['Type'] #Test and Train Data set identifier
num_cols= list(set(list(data.columns))-set(ID_col)-set(target_col)-set(other_col))

In [8]:
#Target variable is also a categorical so convert it
data["target"] =data["target"].astype('str')

train=data[data['Type']=='Train']
test=data[data['Type']=='Test']

train['is_train'] = np.random.uniform(0, 1, len(train)) <= .75
Train, Validate = train[train['is_train']==True], train[train['is_train']==False]
 

C:\Users\AJIT\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [9]:
features=list(set(list(data.columns))-set(ID_col)-set(target_col)-set(other_col))

In [10]:
x_train = Train[list(features)].values
y_train = Train["target"].values
x_validate = Validate[list(features)].values
y_validate = Validate["target"].values
x_test=test[list(features)].values

#### Feature Scaling

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_validate = sc.transform(x_validate)
x_test = sc.transform(x_test)

### ANN

In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense

C:\Users\AJIT\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
classifier = Sequential()

#####  Adding the input layer and the first hidden layer

In [12]:
classifier.add(Dense(units = 100, kernel_initializer = 'uniform', activation = 'relu', input_dim = 200))

#### Adding the second hidden layer

In [13]:
classifier.add(Dense(units = 100, kernel_initializer = 'uniform', activation = 'relu'))

#### Adding the output layer

In [14]:
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

#### Compiling the ANN

In [15]:
import tensorflow as tf
from keras import backend as K
def auc(y_train, y_validate):
    auc = tf.metrics.auc(y_train, y_validate)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=[auc])

#### Fitting the ANN to the Training set

In [16]:
classifier.fit(x_train, y_train, batch_size = 100, epochs = 10)

Epoch 1/100
150268/150268 [==============================] - 12s 81us/step - loss: 0.2538 - auc: 0.7931
Epoch 2/100
150268/150268 [==============================] - 8s 51us/step - loss: 0.2334 - auc: 0.8411
Epoch 3/100
150268/150268 [==============================] - 8s 51us/step - loss: 0.2266 - auc: 0.8496
Epoch 4/100
150268/150268 [==============================] - 8s 52us/step - loss: 0.2206 - auc: 0.8542
Epoch 5/100
150268/150268 [==============================] - 8s 52us/step - loss: 0.2146 - auc: 0.8580
Epoch 6/100
150268/150268 [==============================] - 8s 52us/step - loss: 0.2075 - auc: 0.8612
Epoch 7/100
150268/150268 [==============================] - 7s 46us/step - loss: 0.1999 - auc: 0.8647
Epoch 8/100
150268/150268 [==============================] - 7s 48us/step - loss: 0.1917 - auc: 0.8683
Epoch 9/100
150268/150268 [==============================] - 8s 52us/step - loss: 0.1835 - auc: 0.8721
Epoch 10/100
150268/150268 [==============================] - 8s 53us/st

### Prediction

In [17]:
probs = classifier.predict_proba(x_validate)
# calculate AUC
auc = roc_auc_score(y_validate, probs)
print('AUC: %.3f' % auc)

AUC: 0.639


# Summary

#### Pre-processing, feature engineering, data analysis, Cross Validation, and hyperparameter tuning avoided due to less computing resources.